In [2]:
import cv2
from numpy import array

from keras.layers import Activation, BatchNormalization, Conv2D
from keras.models import Model
# from keras.layers import 



Using TensorFlow backend.


### Data preprocessing
HR and LR images in the form of a numpy array from a given list of images.

Low resolution images are down scaled by 4.


In [3]:
def hr_images(images):
    images_hr = array(images)
    return images_hr

def lr_images(images_real, downscale):
    images = []
    for img in range(len(images_real)):
        images.append(cv2.resize(images_real[img], [images_real[img].shape[0]//downscale, images_real[img].shape[1]//downscale], inter='bicubic', mode = None))
    images_lr = array(images)
    return images_lr


### Generator Network
Number of Residual blocks used are 16.

Number of up-sampling blocks are 2.


In [5]:
def res_block_gen(model, kernel_size, filters, strides):
 
    generator = model
    
    generator.add(Conv2D(filters, kernel_size = kernel_size, filters = filters, strides = strides, padding = 'same'))
    generator.add(BatchNormalization(momentum = 0.5))
    generator.add(PReLU(alpha_initializer = "zeros", alpha_regularizer = None, alpha_constraint = None, shared_axes=[1,2]))
    generator.add(Conv2D(filters, kernel_size = kernel_size, filters = filters, strides = strides, padding = 'same'))
    generator.add(BatchNormalization(momentum = 0.5))
    
    return generator

def up_sampling_block(model, kernel_size, filters, strides):
    
    model.add(Conv2D(filters, kernel_size = kernel_size, filters = filters, strides = strides, padding = 'same'))
    model.add(UpSampling(size = 2))
    model.add(LeakyRelU(alpha = 0.2))
    
    return model


class Generator(object):
    
    def __init__(self, noise_shape):
        self.noise_shape = noise_shape
        
    def generator(self):
        gen_input = Input(shape = self.noise_shape)
        
        model = gen_input.add(Conv2D(filters = 64, kernel_size = 9, strides = 1, padding = "same"))
        model.add(PReLU(alpha_initializers = 'zeros', alpha_regularizer = None, shared_axes = [1,2] ))
        
        gen_model = model
        
        for index in range(16):
            model = res_block_gen(model, 3, 64, 1)
            
        model.add(Conv2D(filters = 64, kernel_size =3, strides = 1, padding = "same"))
        model.add(BatchNormalization(momentum = 0.5))
        
        model.add(gen_input)
        
        for index in range(2):
            model = up_smapling_block(model, 3, 256, 1)
                
        model.add(Conv2D(filters = 64, kernel_size =9, strides = 1, padding = "same"))
        model.add(Activation('tanh'))
        
        generator_model = Model(inputs = gen_input, outputs = model)
        
        return generator_model


### Discriminator Network


In [4]:
''' Discriminator is use to distinguish the HR images and back-propagate the GAN loss to train the discriminator and the generator'''

def discriminator_block(model, filters, kernel_size, strides):
    
    model.add(Conv2D(filters, kernel_size = kernel_size, strides = strides, padding = 'same'))
    model.add(BatchNormalization(momentum = 0.5))
    model.add(LeakyRelU(alpha = 0.2))
    
    return model

class Discriminator(object):
    
    def __init__(self, image_shape):
        self.image_shape = image_shape
        
    def Discriminator(self):
        
        dis_input = Input(shape = self.image_shape)
        
        model = dis_input.add(Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same"))
        model.add(LeakyReLU(alpha = 0.2))
        
        model = discriminator_block(model, 64, 3, 2)
        model = discriminator_block(model, 128, 3, 1)
        model = discriminator_block(model, 128, 3, 2)
        model = discriminator_block(model, 256, 3, 1)
        model = discriminator_block(model, 256, 3, 2)
        model = discriminator_block(model, 512, 3, 1)
        model = discriminator_block(model, 512, 3, 2)

        model.add(Flatten())
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha = 0.2))
        
        model.add(Dense(1))
        model.Activation('sigmoid')
        
        discriminator_model = Model(inputs = dis_input, outputs = model)
        
        return discriminator_model

### Content Loss Function

Compares the outputs of the first convolutionns of VGG.

This loss ensures the GAN model is oriented towards a deblurring task.

In [6]:
 def vgg_loss(self, y_true, y_pred):
        
        vgg19 = VGG19(include_top = False, weights = "imagenet", input_shape = self.image_shape)
        vgg19.trainable = False
        
        for l in vgg19.layers:
            l.trainable = False
        
        loss_model = Model(inputs = vgg19.input, outputs = vgg19.get_layer('block5_conv4').output)
        loss_model.trainable = False
        
'''vgg_loss here represents the MSE(Mean Square Error) of features extracted by a VGG-19 network. For a 
   specific layer within VGG-19, we want their features to be matched (Minimum MSE for features)'''
       
        vgg_loss = K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))
        return vgg_loss

### Generator and Discriminator combined
Two loss used, content loss defined above and Adversarial loss(Binary cross-entropy loss)

In [9]:
def get_gan_network(discriminator, shape, generator, optimizer, vgg_loss):
    
    discriminator.trainable = False #Why is the disciminator parameters not to be trained?
    
    gan_input = Input(shape = shape)#Why is here this input?
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs = gan_input, outputs=[x,gan_output])
    #Compiled GAN network with VGG loss and binary crossentropy loss
    gan.compile(loss = [vggloss, "binary_crossentropy"], loss_weights = [1., 1e-3], optimizer = optimizer )
    
    return gan

### Optimizer Function
Adam optimizer with *B1* = 0.9, *B2* = 0.999  and learning rate = 0.0001

In [10]:
def get_optimizer():
    
    adam = Adam(lr = 0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08) 
    return adam

In [ ]:
###